<a href="https://colab.research.google.com/github/AlifiyulAkyun/SparkMLLib/blob/main/movieLens(Slide_30).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama : Alifiyul Akyun
* No : 04
* Kelas : TI3B
* NIM : 2041720036

In [2]:
# Sumber : https://github.com/cloudxlab/bigdata/blob/master/spark/examples/mllib/ml-recommender.scala

In [3]:

# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Install PySpark
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=16c0b3f1b64ebb2a76d351ea5aea6f280cd07449610448ad49ac5ad6f216bb19
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [5]:

# Import Library
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Movie Lens").getOrCreate()

In [6]:
# Parse String Menjadi Objek Rating
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return (int(fields[0]), int(fields[1]), float(fields[2]), int(fields[3]))

In [7]:
# Baca File
raw = spark.read.text("/content/drive/MyDrive/bigdata-master/spark/examples/mllib/ratings.dat").rdd.map(lambda x: x[0])
ratings = raw.map(parseRating).toDF(["userId", "movieId", "rating", "timestamp"])
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|   5.0|978300760|
|     1|    661|   3.0|978302109|
|     1|    914|   3.0|978301968|
|     1|   3408|   4.0|978300275|
|     1|   2355|   5.0|978824291|
+------+-------+------+---------+
only showing top 5 rows



In [8]:
# Data Training 80% dan Test 20%
training, test = ratings.randomSplit([0.8, 0.2])

In [9]:
# Membuat Model
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)
model.save("mymodel")


In [10]:

# Prediksi Data
predictions = model.transform(test)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]
print("Mean Squared Error:", mse)

predictions.show(10)

Mean Squared Error: nan
+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|      1|   5.0|978824268|  4.014726|
|     1|    150|   5.0|978301777| 4.0938435|
|     1|    783|   4.0|978824291|  4.244564|
|     1|   1207|   4.0|978300719| 4.4342365|
|     1|   1246|   4.0|978302091| 4.8211203|
|     1|   1270|   5.0|978300055| 3.9192498|
|     1|   2294|   4.0|978824291| 3.3842287|
|     1|   2321|   3.0|978302205| 3.0804398|
|     1|   2398|   4.0|978302281|  4.376802|
|     1|   2804|   5.0|978300719|  4.485386|
+------+-------+------+---------+----------+
only showing top 10 rows



In [11]:

# Menyimpan Hasil Prediksi
predictions.write.format("csv").save("ml-predictions.csv")